<a href="https://colab.research.google.com/github/sw6820/kostat/blob/main/kostat_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt 

import re

In [ ]:
df_kor = pd.read_excel('/content/drive/MyDrive/stat/한국표준산업분류(10차)_국문.xlsx', )
df_kor

In [ ]:
df_eng = pd.read_excel('/content/drive/MyDrive/stat/한국표준산업분류(10차)_영문.xlsx')
df_eng

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/stat/1. 실습용자료.txt', sep='|', encoding='cp949')
train

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/stat/2. 모델개발용자료.txt', sep='|', encoding='cp949')
test

In [ ]:
for df in [train, test]:  
  print(df.info())
  print(df.describe())

In [ ]:
for col in train.columns[4:]:
  train_len_mean = np.mean(train[col].str.len())
  train_len_max = np.max(train[col].str.len())
  train_len_min = np.min(train[col].str.len())
  print(f'the length of train {col} - (min, mean, max) : ({train_len_min}, {train_len_mean}, {train_len_max})')

In [ ]:
plt.figure(figsize=(20,12))
plt.suptitle("Histogram: the lengths", fontsize=40)
col_name = train.columns[4:]
for idx, col in enumerate(col_name):
  length = [len(str(x)) for x in df[col]]
  plt.subplot(3, 1, idx+1)
  plt.title(col, fontsize=20)
  plt.hist(length, alpha=0.5, color='orange')

plt.show()

In [ ]:
for col in test.columns[4:]:
  test_len_mean = np.mean(test[col].str.len())
  test_len_max = np.max(test[col].str.len())
  test_len_min = np.min(test[col].str.len())
  print(f'the length of test {col} - (min, mean, max) : ({test_len_min}, {test_len_mean}, {test_len_max})')

In [ ]:
plt.figure(figsize=(20,12))
plt.suptitle("Histogram: the lengths", fontsize=40)
col_name = test.columns[4:]
for idx, col in enumerate(col_name):
  length = [len(str(x)) for x in df[col]]
  plt.subplot(3, 1, idx+1)
  plt.title(col, fontsize=20)
  plt.hist(length, alpha=0.5, color='orange')

plt.show()

In [ ]:
char_dict = {'consonant, vowel' : r'[ㄱ-ㅎㅏ-ㅣ]', 'chinese characters' : r'[一-龥㐀-䶵豈-龎]', \
             'numbers' : r'[0-9]', 'english characters' : r'[a-zA-Z]', \
             'japanese characters' : r'[ぁ-ゔァ-ヴー々〆〤]',\
             'special_characters' : r'[^0-9a-zA-Z가-힣一-龥㐀-䶵豈-龎ぁ-ゔァ-ヴー々〆〤ㄱ-ㅎㅏ-ㅣ\t\n ]'}

In [ ]:
def characters(df, s, characters, regex):
  for col in df.columns[s:]:
    p = re.compile(regex)
    find_char = df[col].str.findall(p)
    print(f'{characters} : {col} : {find_char.apply(lambda x: len(str(x))).sum()}')

In [ ]:
for key, val in char_dict.items():
  characters(train, 4, key, val)
  print('--------------------------------------------------')

In [ ]:
special_characters = set()
p=re.compile(r'[^0-9a-zA-Z가-힣一-龥㐀-䶵豈-龎ぁ-ゔァ-ヴー々〆〤ㄱ-ㅎㅏ-ㅣ\t\n ]')
for col in train.columns[4:]:
  special = pd.DataFrame(train[col].str.findall(p))
  special.dropna(inplace=True)
  special.applymap(lambda x: special_characters.update(x))
print(f'special {len(special_characters)} characters : ', special_characters)  

In [ ]:
special_characters = [*special_characters]
for c in special_characters:
  print(c[1:] if '\\' in c else c)
  if c=='\\': continue
  try:
    p=re.compile(r'['+c+']')
  except:    
    p=re.compile(r'[\\'+c+']')
  for col in train.columns[4:]:
    print(f'special characters : {col} : {train[col].apply(lambda x: len(str(x))).sum()}')